In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans



reg_season_detailed_pd = pd.read_csv('data/RegularSeasonDetailedResults_Prelim2018.csv')
reg_season_detailed_pd.columns = ['Season', 'Daynum', 'Wteam', 'Wscore', 'Lteam', 'Lscore', 'Wloc',
       'Numot', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wor', 'Wdr',
       'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3',
       'Lftm', 'Lfta', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf']
reg_season_detailed_pd.columns
seasons_pd = pd.read_csv('data/Seasons.csv')
teams_pd = pd.read_csv('data/Teams.csv')
teamList = teams_pd['Team_Name'].tolist()
tourney_compact_pd = pd.read_csv('data/TourneyCompactResults.csv')
tourney_detailed_pd = pd.read_csv('data/TourneyDetailedResults.csv')
tourney_seeds_pd = pd.read_csv('data/TourneySeeds.csv')
tourney_seeds_pd.columns = ['Season', 'Seed', 'Team_Id']
tourney_slots_pd = pd.read_csv('data/TourneySlots.csv')
ncaa_tourney_compact_pd = pd.read_csv('data/NCAATourneyCompactResults.csv')
teams_pd_2 = pd.read_csv('data/Teams-2.csv')

In [2]:
reg_season_detailed_pd['Wfg%'] = reg_season_detailed_pd['Wfgm'] / reg_season_detailed_pd['Wfga']
reg_season_detailed_pd['Lfg%'] = reg_season_detailed_pd['Lfgm'] / reg_season_detailed_pd['Lfga']
reg_season_detailed_pd['Wfg%3'] = reg_season_detailed_pd['Wfgm3'] / reg_season_detailed_pd['Wfga3']
reg_season_detailed_pd['Lfg%3'] = reg_season_detailed_pd['Lfgm3'] / reg_season_detailed_pd['Lfga3']
reg_season_detailed_pd['Wft%'] = reg_season_detailed_pd['Wftm'] / reg_season_detailed_pd['Wfta']
reg_season_detailed_pd['Lft%'] = reg_season_detailed_pd['Lftm'] / reg_season_detailed_pd['Lfta']

#WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF

In [3]:
reg_season_detailed_2018_pd = reg_season_detailed_pd[reg_season_detailed_pd['Season'] == 2018]

In [4]:
reg_season_detailed_2018_pd.head(3)

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lto,Lstl,Lblk,Lpf,Wfg%,Lfg%,Wfg%3,Lfg%3,Wft%,Lft%
76636,2018,11,1104,82,1272,70,N,0,26,57,...,17,7,3,22,0.456140,0.383333,0.407407,0.117647,0.678571,0.611111
76637,2018,11,1107,69,1233,67,H,0,24,62,...,7,6,3,21,0.387097,0.440678,0.266667,0.181818,0.894737,0.785714
76638,2018,11,1112,101,1319,67,H,0,34,57,...,11,3,1,24,0.596491,0.396552,0.384615,0.307692,0.875000,0.566667


In [5]:
winning_df = reg_season_detailed_2018_pd[['Wteam', 'Wscore', 'Lscore', 'Wfgm', 'Wfga', 'Wfg%', 'Wfga3', 'Wfgm3', 'Wfg%3',
                                          'Wftm', 'Wfta', 'Wft%', 'Wor', 'Wdr', 'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf']]
losing_df = reg_season_detailed_2018_pd[['Lteam', 'Lscore', 'Wscore', 'Lfgm', 'Lfga', 'Lfg%', 'Lfga3', 'Lfgm3', 'Lfg%3',
                                         'Lftm', 'Lfta', 'Lft%', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf']]
winning_df.columns = ['team', 'for', 'against', 'fgm', 'fga', 'fg%', 'fga3', 'fgm3', 'fg%3', 'ftm', 'fta', 'ft%',
                      'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']
losing_df.columns = ['team', 'for', 'against', 'fgm', 'fga', 'fg%', 'fga3', 'fgm3', 'fg%3', 'ftm', 'fta', 'ft%',
                     'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']
season_stats_df = winning_df.append(losing_df)
season_stats = season_stats_df.groupby(['team'], as_index=False).mean()
print(season_stats)

     team        for    against        fgm        fga       fg%       fga3  \
0    1101  71.291667  73.208333  25.541667  56.250000  0.454961  20.291667   
1    1102  66.000000  73.583333  22.416667  55.625000  0.402772  24.125000   
2    1103  72.153846  77.230769  24.923077  57.615385  0.432307  26.384615   
3    1104  73.357143  69.392857  25.821429  55.071429  0.469350  19.392857   
4    1105  59.407407  75.814815  21.111111  54.666667  0.387923  19.740741   
5    1106  68.307692  78.692308  23.884615  58.384615  0.406937  20.538462   
6    1107  73.321429  67.964286  26.035714  56.464286  0.462017  15.250000   
7    1108  66.333333  76.166667  23.000000  55.708333  0.414710  21.916667   
8    1110  64.785714  72.928571  23.035714  53.500000  0.431129  23.535714   
9    1111  73.481481  76.851852  25.444444  59.814815  0.426233  25.629630   
10   1112  81.428571  71.642857  29.107143  57.071429  0.513250  18.428571   
11   1113  83.777778  74.888889  28.074074  60.518519  0.466255 

In [6]:
# perform clustering 
%matplotlib inline

num_clusters = 14
# stats to cluster on 
feature_columns = ['for', 'against', 'fga', 'fg%', 'fga3', 'fg%3', 'fta', 'ft%',
                      'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']

cluster_columns = ['for', 'against', 'ft%', 'fg%3', 'blk', 'to', 'or', 'dr'] #best performing model 
to_cluster = season_stats[cluster_columns]

# normalize the stats 
x = to_cluster.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
scaled_df = pd.DataFrame(x_scaled)
scaled_df.columns = cluster_columns

# fit kmeans using to_cluster (non-normalized) or scaled_df(normalized)
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(to_cluster)

# extract data about clusters
cluster_map = pd.DataFrame()
cluster_map['Team_Id'] = season_stats.team.values
cluster_map['cluster'] = kmeans.labels_
pairs = []

#create matrix for cluster vs cluster percentages
for i in range(0,num_clusters):
    for j in range(0,num_clusters):
        if(i != j):
            pairs.append([i, j])

percentages = np.zeros((num_clusters,num_clusters))
for pair in pairs:
    cluster_1_wins = reg_season_detailed_2018_pd[
        reg_season_detailed_2018_pd['Wteam'].isin(cluster_map[cluster_map['cluster'] == pair[0]].Team_Id) & 
        reg_season_detailed_2018_pd['Lteam'].isin(cluster_map[cluster_map['cluster'] == pair[1]].Team_Id)
    ]
    cluster_2_wins = reg_season_detailed_2018_pd[
        reg_season_detailed_2018_pd['Wteam'].isin(cluster_map[cluster_map['cluster'] == pair[1]].Team_Id) & 
        reg_season_detailed_2018_pd['Lteam'].isin(cluster_map[cluster_map['cluster'] == pair[0]].Team_Id)
    ]
    percentages[pair[0], pair[1]] = cluster_1_wins.shape[0] / (cluster_1_wins.shape[0] + cluster_2_wins.shape[0])
    percentages[pair[1], pair[0]] = cluster_2_wins.shape[0] / (cluster_1_wins.shape[0] + cluster_2_wins.shape[0])

# get cluster centroids
centers = pd.DataFrame(kmeans.cluster_centers_)
centers.columns = cluster_columns
centers['cluster'] = ['cluster ' + str(i) for i in centers.index]
centers.set_index(['cluster'])
print(centers)
print(pd.DataFrame(percentages))

# get list of teams in each 
mm_teams = tourney_seeds_pd[tourney_seeds_pd['Season'] == 2018].Team_Id.values
mm_team_clusters = cluster_map[cluster_map['Team_Id'].isin(mm_teams)]
cluster_seeds = pd.merge(mm_team_clusters, tourney_seeds_pd[tourney_seeds_pd['Season'] == 2018])
team_cluster_seeds = pd.merge(cluster_seeds, teams_pd)
test = team_cluster_seeds.groupby('cluster')['Team_Name'].apply(lambda x: "%s" % ', '.join(x))
print(cluster_map.groupby(['cluster']).count())

          for    against       ft%      fg%3       blk         to        or  \
0   69.061198  70.512987  0.686129  0.336065  3.320063  13.021915  8.808440   
1   74.037011  78.217025  0.713795  0.346547  2.894784  13.010290  8.534309   
2   77.188853  68.708989  0.725596  0.367878  3.729916  11.871223  8.942754   
3   73.951691  84.280721  0.702306  0.342110  2.437069  13.022800  8.901542   
4   73.121576  73.251840  0.716501  0.354665  3.194519  12.706656  8.258414   
5   81.870769  92.609267  0.712902  0.340694  3.190476  14.323626  9.130000   
6   83.809151  73.007545  0.735465  0.379390  4.026286  11.681503  9.312358   
7   73.448748  65.787572  0.708988  0.358334  4.054149  11.825469  8.955712   
8   66.421258  65.456468  0.718685  0.338721  3.456224  12.213663  8.497678   
9   81.957067  79.780478  0.725663  0.360975  4.071588  13.150799  9.182667   
10  64.050301  72.021527  0.691038  0.330826  2.877924  13.063049  7.446456   
11  68.995656  76.689756  0.695112  0.342645  2.9936

In [7]:
#logreg model 

# construct feature matrix, and y vector 
feature_columns = ['for', 'against', 'fga', 'fg%', 'fga3', 'fg%3', 'fta', 'ft%',
                      'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']
team_feature_columns = []
for col in feature_columns:
    team_feature_columns.append(col + '1')
for col in feature_columns:
    team_feature_columns.append(col + '2')
feature_matrix = season_stats[feature_columns]
cluster_feature_columns = []
for pair in pairs:
    cluster_feature_columns.append(",".join(str(x) for x in pair))

full_feature_columns = team_feature_columns + cluster_feature_columns
outcome = []
feature_matrix = []
diff_feature_matrix = []



for game in reg_season_detailed_2018_pd.itertuples():
    if (game.Lteam in cluster_map['Team_Id'].values):
        wteam_id = game.Wteam
        lteam_id = game.Lteam
        wteam_cluster = cluster_map[cluster_map['Team_Id'] == wteam_id].cluster.values[0]
        lteam_cluster = cluster_map[cluster_map['Team_Id'] == lteam_id].cluster.values[0]
        wteam_features = season_stats[season_stats['team'] == wteam_id][feature_columns].values
        lteam_features = season_stats[season_stats['team'] == lteam_id][feature_columns].values

    row1_features = np.append(wteam_features,lteam_features)
    row1_cluster = np.zeros((len(cluster_feature_columns), 1))
    cluster_str1 = str(wteam_cluster) + "," + str(lteam_cluster)
    if (wteam_cluster != lteam_cluster):
        row1_cluster[cluster_feature_columns.index(cluster_str1)] = 1
    row1_features = np.append(row1_features, row1_cluster)
    feature_matrix.append(row1_features)
    diff_feature_matrix.append(np.subtract(wteam_features,lteam_features)[0])
    outcome.append(1)
    
    row2_features = np.append(lteam_features,wteam_features)
    row2_cluster = np.zeros((len(cluster_feature_columns), 1))
    cluster_str2 = str(lteam_cluster) + "," + str(wteam_cluster)
    if (wteam_cluster != lteam_cluster):
        row2_cluster[cluster_feature_columns.index(cluster_str2)] = 1
    row2_features = np.append(row2_features, row2_cluster)
    feature_matrix.append(row2_features)
    diff_feature_matrix.append(np.subtract(lteam_features,wteam_features)[0])
    outcome.append(0)

In [8]:
print(feature_matrix[:3])

[array([ 73.35714286,  69.39285714,  55.07142857,   0.46934973,
        19.39285714,   0.32130648,  22.75      ,   0.67187931,
         8.35714286,  25.14285714,  12.85714286,  13.92857143,
         6.57142857,   5.85714286,  19.        ,  70.64285714,
        70.89285714,  53.96428571,   0.44814914,  18.64285714,
         0.30815122,  23.82142857,   0.68491813,   9.96428571,
        21.32142857,  13.03571429,  13.42857143,   6.21428571,
         4.07142857,  16.71428571,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0. 

In [9]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import numpy as np

logreg = LogisticRegression()
features_rfe = []; 
features_rfe_not = [];
rfe = RFE(logreg, 199)
rfe = rfe.fit(feature_matrix,outcome)
for i in range(len(rfe.support_)):
    if np.any(rfe.support_[i]):
        features_rfe.append(full_feature_columns[i])

for i in range(len(rfe.support_)):
    if not(np.any(rfe.support_[i])):
        features_rfe_not.append(full_feature_columns[i])
        
print(rfe.support_)
print(features_rfe)
print(features_rfe_not)
#features_rfe = pd.DataFrame(rfe.support_[0])
#features_rfe.columns = full_feature_columns

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[ True  True False  True False  True  True  True False False  True  True
  True  True  True  True  True False  True False  True  True  True False
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True False  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  T

In [10]:
#run logreg
custom_columns = ['for1', 'fg%1', 'fg%31', 'or1', 'dr1', 
                  'for2', 'fg%2', 'fg%32', 'or2', 'dr2']

features_df = pd.DataFrame(feature_matrix)
features_df.columns = full_feature_columns
sub_features_matrix = features_df[features_rfe].values

#test_feature_pd = pd.DataFrame(test_feature_matrix)
#test_feature_pd.columns = full_feature_columns
#sub_test_feature_matrix = test_feature_pd[full_feature_columns].values

# diff_features_df = pd.DataFrame(diff_feature_matrix)
# diff_features_df.columns = feature_columns
# diff_sub_features_matrix = diff_features_df[feature_columns].values

# diff_test_feature_pd = pd.DataFrame(diff_test_feature_matrix)
# diff_test_feature_pd.columns = feature_columns
# diff_sub_test_feature_matrix = diff_test_feature_pd[feature_columns].values

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression

x_train, x_test, y_train, y_test = train_test_split(sub_features_matrix, outcome, test_size=0.33, random_state=42)
logreg = LogisticRegression(C=100, penalty='l1', random_state=42)
logreg.fit(sub_features_matrix, outcome)
y_pred = logreg.predict(x_test)
print('Accuracy of logistic regression classifier on test set: {:.10f}'.format(logreg.score(sub_features_matrix, outcome)))

Accuracy of logistic regression classifier on test set: 0.7502646623


In [19]:
matchups = pd.read_csv('data/SampleSubmissionStage2.csv')
d = ['for', 'against', 'fg%', 'fg%3', 'fta', 'ft%', 'ast', 'to', 'stl', 'blk', 'pf']
c = []
for pair in cluster_feature_columns:
    if pair in features_rfe:
        c.append(pair)

list_matchups = []
for game in matchups.itertuples():
    year,team1_id,team2_id = game.ID.split("_")
    team1_cluster = cluster_map[cluster_map['Team_Id'] == int(team1_id)].cluster.values[0]
    team2_cluster = cluster_map[cluster_map['Team_Id'] == int(team2_id)].cluster.values[0]
    team1_features = season_stats[season_stats['team'] == int(team1_id)][d].values
    team2_features = season_stats[season_stats['team'] == int(team2_id)][d].values
    
    row_features = np.append(team1_features,team2_features)
    row_cluster = np.zeros((len(c), 1))
    cluster_str = str(team1_cluster) + "," + str(team2_cluster)
    if (team1_cluster != team2_cluster and cluster_str in c):
        row_cluster[c.index(cluster_str)] = 1
    row_features = np.append(row_features, row_cluster)
    prob = logreg.predict_proba([row_features])[0][1]
    list_matchups.append([game.ID, prob])

submission = pd.DataFrame(list_matchups, columns=matchups.columns)
submission.to_csv('data/Submission2018.csv', index=False)

In [270]:
len(full_feature_columns)

212

In [ ]:
logreg.predict(sub_features_matrix)
proba = logreg.predict_proba(sub_features_matrix)
diff_arr = logreg.predict(sub_features_matrix) - outcome
y_pred_proba = []
for index, game in enumerate(tourney_games.itertuples()):z
    if diff_arr[index] != 0:
        team_1 = teams_pd[teams_pd['Team_Id'] == game.WTeamID]['Team_Name'].values[0]
        team_2 = teams_pd[teams_pd['Team_Id'] == game.LTeamID]['Team_Name'].values[0]
        print(team_1, team_2, logreg.predict_proba(sub_features_matrix)[index])
    y_pred_proba.append(proba[index][0])
print(test_outcome)